In [22]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout

In [23]:
from ISLP import load_data
import numpy as np
import siuba
from siuba import _, if_else, select, mutate
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
import pandas as pd

In [24]:
smarket = load_data('Smarket')

In [25]:
smarket2=(
    smarket >>
    #mutate(Direction = if_else(_.Direction == 'Up',1,0)) >>
    mutate(train = if_else(_.Year < 2005,1,0) ) >>
    select(~_.Year,~_.Today)

)
train = (smarket2.train == 1)
# print(smarket2.head())

# print(design)

X = (
    smarket2 >>
    select(~_.train,~_.Direction)
)
y = smarket2['Direction']  == 'Up'

X_train, X_test = X.loc[train], X.loc[~train]
Y_train, Y_test = y.loc[train], y.loc[~train]
Y_test_raw =Y_test

In [26]:
X_train = (
    X_train# >>    select(_.Lag1,_.Lag2)

)
X_test = (
    X_test #>>    select(_.Lag1,_.Lag2)

)
X_train.columns

Index(['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume'], dtype='object')

In [27]:
Y_test

998     False
999     False
1000    False
1001     True
1002    False
        ...  
1245     True
1246    False
1247     True
1248    False
1249    False
Name: Direction, Length: 252, dtype: bool

In [28]:
# X_train
feature_vector_length = 6
num_classes = 2

In [46]:
to_categorical([1,0,1], 2)

array([[0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [29]:
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model


Feature shape: (6,)


In [51]:
print(Y_test[:])
y.loc[~train]

[1. 0.]


998     False
999     False
1000    False
1001     True
1002    False
        ...  
1245     True
1246    False
1247     True
1248    False
1249    False
Name: Direction, Length: 252, dtype: bool

In [31]:
batch_size = 250
hidden_units = 2
dropout = 0.45

In [32]:
# best model
model = Sequential()
model.add(Dense(256, input_shape=input_shape, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(num_classes, activation='softmax'))
#Test results - Loss: 0.6916869282722473 - Accuracy: 0.5357142686843872%

In [33]:
# best model
model = Sequential()
model.add(Dense(64, input_shape=input_shape, activation='relu'))
model.add(Dense(256, input_shape=input_shape, activation='relu'))
model.add(Dense(64, input_shape=input_shape, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))
#Test results - Loss: 0.7014935612678528 - Accuracy: 0.5595238208770752%

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=15, batch_size=batch_size, verbose=1, validation_split=0.2)

# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Epoch 1/15
4/4 [==============================] - 1s 94ms/step - loss: 0.6970 - accuracy: 0.4699 - val_loss: 0.6930 - val_accuracy: 0.5200
Epoch 2/15
4/4 [==============================] - 0s 13ms/step - loss: 0.6907 - accuracy: 0.5401 - val_loss: 0.7000 - val_accuracy: 0.4450
Epoch 3/15
4/4 [==============================] - 0s 11ms/step - loss: 0.6887 - accuracy: 0.5363 - val_loss: 0.6991 - val_accuracy: 0.4700
Epoch 4/15
4/4 [==============================] - 0s 12ms/step - loss: 0.6839 - accuracy: 0.5664 - val_loss: 0.6963 - val_accuracy: 0.4500
Epoch 5/15
4/4 [==============================] - 0s 11ms/step - loss: 0.6805 - accuracy: 0.5877 - val_loss: 0.6948 - val_accuracy: 0.4800
Epoch 6/15
4/4 [==============================] - 0s 12ms/step - loss: 0.6780 - accuracy: 0.5627 - val_loss: 0.6961 - val_accuracy: 0.4850
Epoch 7/15
4/4 [==============================] - 0s 11ms/step - loss: 0.6751 - accuracy: 0.5739 - val_loss: 0.7013 - val_accuracy: 0.4850
Epoch 8/15
4/4 [===========

In [35]:
test_results

[0.6898758411407471, 0.5396825671195984]

In [36]:
test_pred_raw = model.predict(X_test)
test_pred = np.argmax(test_pred_raw, axis=1)
test_pred


8/8 [==============================] - 0s 3ms/step


array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1], dtype=int64)

In [41]:
# print(np.array(Y_test))
# print(test_pred)
# print(len(np.array(Y_test)))
# print(len(test_pred))
df1 = pd.DataFrame(
   
    {'pred':  test_pred}
    
)

df2 = (df1 >>
       mutate(y=np.array(y.loc[~train])) >>
       mutate(y=if_else(y,1,0))
)

df2.head()


,pred,y
0,1,1
1,1,1
2,1,0
3,1,1
4,0,1


In [42]:
ct = confusion_table(test_pred, Y_test_raw)
print(ct)

Truth       0   1
Predicted        
0          49  54
1          62  87


In [43]:
type(ct)

pandas.core.frame.DataFrame

In [44]:
TN = ct.iloc[0,0]
TP = ct.iloc[1,1]
FN = ct.iloc[0,1]
FP = ct.iloc[1,0]
Accuracy = (TP + TN) /(TN+TP+FN+FP)
Recall = TP /(TP+FN)
Precision = TP/(TP+FP)
F_score = Recall*Precision/(Recall +Precision)
df_r = pd.DataFrame(
 {'Accuracy': [Accuracy]
  ,  'Precision':[Precision],   'Recall':[Recall],    'F_score':[F_score]
 }
)
df_r

,Accuracy,Precision,Recall,F_score
0,0.539683,0.583893,0.617021,0.3
